#**VOICE CONVERSION (ML PROJECT PHASE 2) - GROUP 30**

##**PIPELINE**

##**SETTING UP THE SYSTEM**

In [ ]:
!pip install pydub

In [ ]:
!pip install fastdtw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fastdtw: filename=fastdtw-0.3.4-cp310-cp310-linux_x86_64.whl size=512408 sha256=86bcff6c1e0c3c323760caca2645bfff9ad4a4f13aae974502550e8f878e7b12
  Stored in directory: /root/.cache/pip/wheels/73/c8/f7/c25448dab74c3acf4848bc25d513c736bb93910277e1528ef4
Successfully built fastdtw


In [ ]:
import os
import string
import librosa
import librosa.display

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from google.colab import drive
from sklearn.preprocessing import LabelEncoder
from pydub import AudioSegment
from IPython.display import Audio
from scipy.spatial.distance import cdist
from scipy import signal
from scipy.signal import medfilt
from fastdtw import fastdtw

In [ ]:
import pandas as pd

# Specify the path to your CSV file
csv_file_path = '/content/sample_data/common_sentences_paths.csv'

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

In [ ]:
c1=df['c1_path'].tolist()
c2=df['c2_path'].tolist()

In [ ]:
import os
import librosa

def load_audio_files(folder_path,audioList):
    """
    Load all audio files from a folder.

    Parameters:
    - folder_path (str): Path to the folder containing audio files.

    Returns:
    - audio_files (list): List of tuples containing audio data and sampling rate.
    """

    audio_files = []

    # Iterate through files in the folder
    for file_name in audioList:
        file_path = os.path.join(folder_path, file_name)

        # Check if the file is an audio file (you may want to add more file format checks)
        if file_name.lower().endswith(('.wav', '.mp3', '.ogg')):
            # Load audio file using librosa
            audio, sr = librosa.load(file_path, sr=None)
            audio_files.append((audio, sr))

    return audio_files

# Example usage:
folder_path = "/content/sample_data/Audio"
loaded_audio_source = load_audio_files(folder_path,c1)
loaded_audio_target = load_audio_files(folder_path,c2)




In [ ]:
print(len(loaded_audio_source))

54


##**1. FEATURE EXTRACTION**

In [ ]:
def extractFeatures(y,sr):

  # Compute the spectrogram
  spectrogram = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)

  # Calculate audio duration
  audio_duration = len(y) / sr

  # Print data
  print("Audio duration: ", audio_duration)

  # Display the audio player
  display(Audio(data=y, rate=sr))

  # Plot the spectrogram for the the pre-emphasized waveform
  plt.figure(figsize=(10, 6))
  librosa.display.specshow(spectrogram, sr=sr, x_axis='time', y_axis='log')
  plt.colorbar(format='%+2.0f dB')
  plt.title('Original Spectrogram')

  return spectrogram

In [ ]:
spectrogram_data_source = []
spectrogram_data_target = []

for i in range(54):
  spectrogram_data_source.append(extractFeatures(loaded_audio_source[i][0],loaded_audio_source[i][1]))
  spectrogram_data_target.append(extractFeatures(loaded_audio_target[i][0],loaded_audio_target[i][1]))

In [ ]:
def extract_mfcc(audio_path, n_mfcc=13, hop_length=512, n_fft=2048):
    """
    Extracts MFCC features from an audio file.

    Parameters:
    - audio_path (str): Path to the audio file.
    - n_mfcc (int): Number of MFCC coefficients to extract.
    - hop_length (int): Hop length for the STFT (Short-Time Fourier Transform).
    - n_fft (int): Number of samples in each window of the STFT.

    Returns:
    - mfcc_features (numpy.ndarray): Extracted MFCC features.
    """

    # Load audio file
    y, sr = librosa.load(audio_path, sr=None)

    # Extract MFCC features
    mfcc_features = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, hop_length=hop_length, n_fft=n_fft)
    delta_features = librosa.feature.delta(mfcc_features)
    features = np.concatenate((mfcc_features,delta_features))
    return mfcc_features

In [ ]:
def reconstruct_audio(spectrogram):
  # Reconstruct the audio from the spectrogram
  y_reconstructed = librosa.griffinlim(librosa.db_to_amplitude(spectrogram))
  return y_reconstructed

##**2. FEATURE ALLIGNMENT**

In [ ]:
def alignFeatures(source_features, target_features):
    # Calculate lengths of both features
    source_len = source_features.shape[1]
    target_len = target_features.shape[1]
    print(source_len)
    # Identify the longer audio and calculate the difference in lengths
    if source_len > target_len:
        padding = source_len - target_len
        # Pad the target features
        target_features = np.pad(target_features, ((0, 0), (0, padding)), mode='constant')
    elif target_len > source_len:
        padding = target_len - source_len
        # Pad the source features
        source_features = np.pad(source_features, ((0, 0), (0, padding)), mode='constant')

    # Calculate pairwise distances between source and target MFCCs
    distance_matrix = cdist(source_features.T, target_features.T, metric='euclidean')

    # Apply dynamic time warping
    alignment_path = fastdtw(source_features.T, target_features.T)[1]  # Using the features themselves as inputs

    # Extract aligned frames using the alignment path
    aligned_source = [source_features[:, i] for i, j in alignment_path]
    aligned_target = [target_features[:, j] for i, j in alignment_path]

    # Convert lists to arrays
    aligned_source = np.array(aligned_source).T
    aligned_target = np.array(aligned_target).T

    return aligned_source, aligned_target

In [ ]:
aligned_source_data = []
aligned_target_data = []

for i in range(54):
    d1, d2 = alignFeatures(spectrogram_data_source[i], spectrogram_data_target[i])
    aligned_source_data.append(d1)
    aligned_target_data.append(d2)

183
417
388
565
466
502
385
151
518
149
559
541
363
466
426
388
374
165
430
419
349
543
455
291
381
448
444
590
518
406
487
482
174
457
397
343
570
318
295
496
520
356
190
172
160
653
570
176
640
439
460
419
138
165


In [ ]:
print(len(list(aligned_target_data[0][0])))

224


##**4. CONVERSION**

In [ ]:
import numpy as np

def make_features_compatible(matrix_list, target_num_features):
    """
    Make the number of features in all matrices in the list compatible with a target number of features.

    Parameters:
    - matrix_list (list): List of matrices, where each matrix is a 2D NumPy array.
    - target_num_features (int): Target number of features that you want all matrices to have.

    Returns:
    - modified_matrices (list): List of matrices with the number of features made compatible with the target.
    """
    modified_matrices = []

    for matrix in matrix_list:
        # Reshape or pad/truncate the matrix to match the target number of features
        if matrix.shape[1] == target_num_features:
            modified_matrix = matrix
        elif matrix.shape[1] > target_num_features:
            # Truncate if the matrix has more features than the target
            modified_matrix = matrix[:, :target_num_features]
        else:
            # Pad with zeros if the matrix has fewer features than the target
            pad_width = [(0, 0), (0, target_num_features - matrix.shape[1])]
            modified_matrix = np.pad(matrix, pad_width, mode='constant', constant_values=0)

        modified_matrices.append(modified_matrix)

    return modified_matrices


target_num_features = 100

# Make features compatible
modified_source = make_features_compatible(aligned_source_data, target_num_features)
modified_target = make_features_compatible(aligned_target_data, target_num_features)




In [ ]:
import numpy as np
from sklearn.decomposition import PCA

def reduce_dimension(input_matrix, target_num_features):
    """
    Reduce the dimensionality of the input matrix to the specified number of features using PCA.

    Parameters:
    - input_matrix (numpy.ndarray): Input matrix with varying number of features.
    - target_num_features (int): Target number of features for the reduced matrix.

    Returns:
    - reduced_matrix (numpy.ndarray): Matrix with the reduced number of features using PCA.
    """
    # Determine the number of components based on the target number of features
    num_components = min(target_num_features, input_matrix.shape[1])

    # Apply PCA
    pca = PCA(n_components=num_components)
    reduced_matrix = pca.fit_transform(input_matrix)

    return reduced_matrix

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

# Assuming X_source and X_target are matrices of acoustic features
# X_source represents features from the source speaker
# X_target represents features from the target speaker

# Learn a linear transformation using a simple linear regression model
model = LinearRegression()
model.fit(aligned_source_data[1], aligned_target_data[1])
# Given new source features, transform them to target features

# Make sure aligned_source_data[i] has the same number of features as during training
converted_feature = model.predict(aligned_source_data[1])



##**8. EVALUATION**

In [ ]:
# Inverse MFCC transformation to reconstruct audio
converted_audio = reconstruct_audio(converted_feature)

# Play the reconstructed audio
display(Audio(data=converted_audio, rate=loaded_audio_target[1][1]))

In [ ]:
converted_audio == aligned_target_data

<ipython-input-160-30068bcd3b0a>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  converted_audio == aligned_target_data
<ipython-input-160-30068bcd3b0a>:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  converted_audio == aligned_target_data


False

#NMF

In [ ]:
import numpy as np
import librosa
from sklearn.decomposition import DictionaryLearning
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

# Step 1: Load and Prepare Data

# Assuming you have audio files for the source and target speakers
source_audio_path = "common_voice_en_203759.mp3"
target_audio_path = "common_voice_en_614081.mp3"

# Load audio files
source_audio, _ = librosa.load(source_audio_path, sr=None)
target_audio, _ = librosa.load(target_audio_path, sr=None)

# Extract spectrograms
def extract_spectrogram(audio):
    spectrogram = np.abs(librosa.stft(audio))
    return spectrogram

source_spectrogram = extract_spectrogram(source_audio)
target_spectrogram = extract_spectrogram(target_audio)

# Step 2: Time Alignment using DTW

# Use dynamic time warping (DTW) for time alignment
distance, path = fastdtw(source_spectrogram.T, target_spectrogram.T, dist=euclidean)

# Apply time alignment to source spectrogram
source_spectrogram_aligned = np.array([source_spectrogram.T[j] for j in path])

# Flatten the aligned spectrograms
X_source = source_spectrogram_aligned.reshape(1, -1)
X_target = target_spectrogram.reshape(1, -1)

# Step 3: Dictionary Learning

# Set the number of components for dictionary learning
n_components = 100

# Dictionary Learning for source and target
dl_source = DictionaryLearning(n_components=n_components)
dl_target = DictionaryLearning(n_components=n_components)

X_source_transformed = dl_source.fit_transform(X_source)
X_target_transformed = dl_target.fit_transform(X_target)
# print(X_source_transformed.shape)
# print(X_target_transformed.shape)
# Step 4: Transform Source to Target

print(X_source.shape)
# Use the learned dictionaries to transform the source towards the target
X_source_transformed_to_target = dl_target.transform(X_source)

# Step 5: Reconstruction using librosa.griffinlim


